# 4. Optimization in Python

## Problem 2. Workforce planning
---

## Mathematical Formulation

### Decision variables
*   $X_{t}$: Number of untrained programmers to be trained in month t.
*   $Y_{t}$: Number of untrained programmers to be laid off in month t.
*   $U_{t}$: Number of untrained programmers working in production in month t.
*   $T_{t}$: Number of trained programmers working in production in month t.

### Objective function

* Minimize the total cost from January(1st) to June(6th). The labor cost consists of salaries for both untrained and trained programmers.

\begin{aligned}
\textrm{(Minimize)} \quad & \sum\limits_{i=1}^{6} (3000 \times U_i + 3300 \times T_i) \; & & & 
\end{aligned}

### Constraints

\begin{aligned}
\textrm{(Workforce Demand)} \quad U_t + 1.2 \times T_t & \geq Demand_t & \forall t \\
\end{aligned}


\begin{aligned}
\textrm{(Workforce in 1st month)} \quad U_1 + X_1 + Y_1 & = 145 & \\
\end{aligned}

\begin{aligned}
\textrm{(Workforce Alteration)} \quad U_{t+1} = U_t - X_t - Y_t - 0.05 \times U_t \quad & \forall t \\
\quad T_{t+1} = T_t + 0.9 \times X_t \quad & \forall t \\
\end{aligned}


\begin{aligned}
\textrm{(Non-negative integers)} \quad & X_{t},Y_{t},U_{t},T_{t} & \geq  \quad 0, & &\forall t \\
\end{aligned}

---

## Optimization using PuLP

### Step 1: Setup

#### Import required packages

In [1]:
import numpy as np
import pandas as pd
from pulp import *

### Step 2 Create a LP Minimization problem

In [2]:
prob2 = LpProblem("Software_Company_Problem", LpMinimize)

### Step 3 Add decision variables

In [3]:
# Decision Variables

months = ['Jan','Feb','Mar','Apr','May','Jun']

X = LpVariable.dicts("Train", months, lowBound=0, cat="Continuous")
Y = LpVariable.dicts("LayOff", months, lowBound=0, cat="Continuous")
U = LpVariable.dicts("Untrained", months, lowBound=0, cat="Continuous")
T = LpVariable.dicts("Trained", months, lowBound=0, cat="Continuous")

### Step 4 Add the objective function and constraints

In [4]:
# Objective function
unit_cost_untrained = 3000
unit_cost_trained = 3300

prob2 += lpSum([unit_cost_untrained*U[month] + unit_cost_trained*T[month] for month in months]), "Total Cost"

# Constraints

## Contraint1: Workforce Demand
demand_t = {'Jan': 100, 'Feb': 100,  'Mar': 115,  'Apr': 125,  'May': 140,  'Jun': 150}
for month in months:
    prob2 += U[month] + 1.2*T[month] - X[month] - Y[month] >= demand_t[month]

## Constraint2: Workforce in the first month
prob2 += X['Jan'] + Y['Jan'] + U['Jan'] == 145  # Initial number of untrained programmer
prob2 += T['Jan'] == 0  # No trained programmers in January
    
## Constraint3: Workforce Dynamics
for i in range(1, len(months)):  # Start from February
    prev_month = months[i-1]
    curr_month = months[i]
        
    prob2 += U[curr_month] == 0.95 * U[prev_month] - Y[prev_month] + 0.1 * X[prev_month] 
    prob2 += T[curr_month] == T[prev_month] + 0.9 * X[prev_month]

In [5]:
# Objective function
unit_cost_untrained = 3000
unit_cost_trained = 3300

prob2 += lpSum([unit_cost_untrained*U[month] + unit_cost_trained*T[month] for month in months]), "Total Cost"

# Constraints

## Contraint1: Workforce Demand
demand_t = {'Jan': 100, 'Feb': 100,  'Mar': 115,  'Apr': 125,  'May': 140,  'Jun': 150}
for month in months:
    prob2 += U[month] + 1.2*T[month] - X[month] - Y[month] >= demand_t[month]

## Constraint2: Workforce in the first month
prob2 += X['Jan'] + Y['Jan'] + U['Jan'] == 145  # Initial number of untrained programmer
prob2 += T['Jan'] == 0  # No trained programmers in January
    
## Constraint3: Workforce Dynamics
for i in range(1, len(months)):  # Start from February
    prev_month = months[i-1]
    curr_month = months[i]
    
    prob2 += U[curr_month] == 0.95 * U[prev_month] - Y[prev_month] + 0.1 * X[prev_month] 
    prob2 += T[curr_month] == T[prev_month] + 0.9 * X[prev_month]

C:\Users\YS Yoon\anaconda3\Lib\site-packages\pulp\pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [6]:
prob2

Software_Company_Problem:
MINIMIZE
3300*Trained_Apr + 3300*Trained_Feb + 3300*Trained_Jan + 3300*Trained_Jun + 3300*Trained_Mar + 3300*Trained_May + 3000*Untrained_Apr + 3000*Untrained_Feb + 3000*Untrained_Jan + 3000*Untrained_Jun + 3000*Untrained_Mar + 3000*Untrained_May + 0
SUBJECT TO
_C1: - LayOff_Jan - Train_Jan + 1.2 Trained_Jan + Untrained_Jan >= 100

_C2: - LayOff_Feb - Train_Feb + 1.2 Trained_Feb + Untrained_Feb >= 100

_C3: - LayOff_Mar - Train_Mar + 1.2 Trained_Mar + Untrained_Mar >= 115

_C4: - LayOff_Apr - Train_Apr + 1.2 Trained_Apr + Untrained_Apr >= 125

_C5: - LayOff_May - Train_May + 1.2 Trained_May + Untrained_May >= 140

_C6: - LayOff_Jun - Train_Jun + 1.2 Trained_Jun + Untrained_Jun >= 150

_C7: LayOff_Jan + Train_Jan + Untrained_Jan = 145

_C8: Trained_Jan = 0

_C9: LayOff_Jan - 0.1 Train_Jan + Untrained_Feb - 0.95 Untrained_Jan = 0

_C10: - 0.9 Train_Jan + Trained_Feb - Trained_Jan = 0

_C11: LayOff_Feb - 0.1 Train_Feb - 0.95 Untrained_Feb + Untrained_Mar = 0

_C1

### Step 5 Solve the problem

In [7]:
#pulpTestAll()  # test solvers
prob2.solve()
#probA.solve(solver=GUROBI_CMD())
print("Status:",LpStatus[prob2.status])

Status: Optimal


### Step 6 Print the optimal solution

In [8]:
for v in prob2.variables():
    print(v.name, "=", v.varValue)
    
print("Total cost=", value(prob2.objective))

LayOff_Apr = 0.0
LayOff_Feb = 0.0
LayOff_Jan = 12.8056
LayOff_Jun = 0.0
LayOff_Mar = 0.0
LayOff_May = 0.0
Train_Apr = 12.1927
Train_Feb = 15.0089
Train_Jan = 9.69443
Train_Jun = 0.0
Train_Mar = 12.4924
Train_May = 6.72912
Trained_Apr = 33.4761
Trained_Feb = 8.72499
Trained_Jan = 0.0
Trained_Jun = 50.5058
Trained_Mar = 22.233
Trained_May = 44.4496
Untrained_Apr = 97.0214
Untrained_Feb = 104.539
Untrained_Jan = 122.5
Untrained_Jun = 89.3931
Untrained_Mar = 100.813
Untrained_May = 93.3896
Total cost= 2348953.6169999996
